In [1]:
import os
openai_api_key = os.environ.get("API_KEY")

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

chat = ChatOpenAI(temperature=.7, openai_api_key=openai_api_key)

In [3]:
chat(
    [
        SystemMessage(content="You are a nice AI bot that helps a user figure out what to eat in one short sentence"),
        HumanMessage(content="I like tomatoes, what should I eat?")
    ]
)


AIMessage(content='How about a Caprese salad with fresh tomatoes, mozzarella cheese, and basil?', additional_kwargs={}, example=False)

In [5]:
response = chat(
    [
        SystemMessage(content="You are a nice AI bot that helps a user figure out where to travel in one short sentence"),
        HumanMessage(content="I like the beaches where should I go?"),
        AIMessage(content="You should go to Nice, France"),
        HumanMessage(content="What else should I do when I'm there?")
    ]
)
response.content

"While you're in Nice, you should also explore the old town, visit the colorful markets, and hike up to the scenic Colline du Château."

In [6]:
from langchain.schema import Document

In [7]:
Document(page_content="This is my document. It is full of text that I've gathered from other places",
         metadata={
             'my_document_id' : 234234,
             'my_document_source' : "The LangChain Papers",
             'my_document_create_time' : 1680013019
         })

Document(page_content="This is my document. It is full of text that I've gathered from other places", metadata={'my_document_id': 234234, 'my_document_source': 'The LangChain Papers', 'my_document_create_time': 1680013019})

In [8]:
chat(
    [
        SystemMessage(content="You are an unhelpful AI bot that makes a joke at whatever the user says"),
        HumanMessage(content="I would like to go to New York, how should I do this?")
    ]
)

AIMessage(content="You should probably walk there, it's great exercise!", additional_kwargs={}, example=False)

In [11]:
import numpy as np
import pandas as pd
import time
from redis.commands.search.field import VectorField
from redis.commands.search.field import TextField
from redis.commands.search.field import TagField
from redis.commands.search.query import Query
import redis

In [37]:
import json
with open("finetuning.json", "r") as f:
    datos = json.loads(f.read())

for d in datos:
    
    if "estado_choices" in d.keys():
        d["completion"] = d["completion"] +" " +json.dumps(d["estado_choices"])

with open("instruccion_db.json", "w") as f:
    f.write(json.dumps(datos))


In [54]:
NUMBER_PRODUCTS=1000

#Load Product data and truncate long text fields
all_prods_df = pd.read_json("instruccion_db.json")


all_prods_df['estado_choices'] = all_prods_df['estado_choices'].apply(json.dumps)
all_prods_df['completion'] = all_prods_df['completion'].apply(json.dumps)
all_prods_df['completion']= all_prods_df["completion"] + " " + all_prods_df["estado_choices"]
all_prods_df['primary_key'] = all_prods_df['prompt'] + '-' + all_prods_df['completion']

all_prods_df.drop("estado_choices",axis=1, inplace=True)
all_prods_df = all_prods_df.astype(str)

all_prods_df.reset_index(drop=True,inplace=True)

#get the first 1000 products with non-empty item keywords
product_metadata = all_prods_df.head(NUMBER_PRODUCTS).to_dict(orient='index')


In [55]:
all_prods_df.tail()

,prompt,completion,primary_key
117,Lista de las 10 últimas mesas que contienen nulos,"""WITH latest_unique_historialnulos AS (SELECT ...",Lista de las 10 últimas mesas que contienen nu...
118,Puedes dadme la informacion completa de la mes...,"""WITH ic AS (SELECT Hora, Fecha, IDCam, UID, C...",Puedes dadme la informacion completa de la mes...
119,Dame toda la informacion de la ultima mesa Z1,"""WITH ic AS (SELECT Hora, Fecha, IDCam, UID, C...",Dame toda la informacion de la ultima mesa Z1...
120,Dame toda la informacion de la ultima Z1 que a...,"""WITH ic AS (SELECT Hora, Fecha, IDCam, UID, C...",Dame toda la informacion de la ultima Z1 que a...
121,dame la informacion de las ultimas 10 mesas,"""WITH ic AS (SELECT Hora, Fecha, IDCam, UID, C...","dame la informacion de las ultimas 10 mesas-""W..."


In [56]:
def load_strings(client, data):
    p = client.pipeline(transaction=False)
    for index, row in data.iterrows():    
        #hash key
        key = 'data:' + str(index)
        
        #hash values
        p.hset(key, mapping=row.to_dict())
            
    p.execute()

def create_index (redis_conn):
    redis_conn.ft().create_index([
        TextField("prompt"),
        TextField("completion")     
    ])


In [100]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

loader = TextLoader('instruccion_db.json')
documents = loader.load()


In [101]:
# Get your splitter ready
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

# Split your docs into texts
texts = text_splitter.split_documents(documents)

# Get embedding engine ready
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

# Embedd your texts
db = FAISS.from_documents(texts, embeddings)

In [102]:
retriever = db.as_retriever()

In [103]:
docs = retriever.get_relevant_documents("Dame todos los camareros activos")

In [104]:
print("\n\n".join([x.page_content[:200] for x in docs[:2]]))

camareros WHERE Activo = 1"}, {"prompt": "Borrar al camarero Bart Simsoms.", "completion": "UPDATE camareros SET Activo = 0 WHERE Nombre = 'Bart' AND Apellidos='Simsoms'; SELECT Nombre, Apellidos, Aut

informaci\u00f3n de los camareros", "completion": "SELECT ID, Nombre, Apellido, Autorizado FROM camareros;"}, {"prompt": "\u00bfCu\u00e1ntos camareros autorizados hay?", "completion": "SELECT COUNT(*)


In [130]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo",temperature=0, openai_api_key=openai_api_key)

In [131]:
# The LangChain component we'll use to get the documents
from langchain.chains import RetrievalQA

docsearch = FAISS.from_documents(docs, embeddings)
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())

In [132]:
qa.run("que consulta seria buena para recuperar la lista de camareros?")

'La consulta para recuperar la lista de camareros sería: "SELECT Nombre, Apellidos, Autorizado FROM camareros WHERE Activo = 1".'

In [133]:
qa.run("puedes crear una consulta sql que busque todos los camareros activos")

'Sí, la consulta SQL para buscar todos los camareros activos sería:\n\nSELECT * FROM camareros WHERE Activo = 1;'

In [134]:
from langchain.prompts import PromptTemplate

# Create our template
template = """
Eres una IA experta en traducir texto a SQL:   

Podrias crar una consulta SQL con la {consulta}. Solo necesito la consulta sin explicaciones.
En una linea.
"""



p = PromptTemplate(
    template=template,
    input_variables=["consulta"]
)

In [135]:
qa.run(p.format(consulta="Dame la lista de camareros") )

'SELECT Nombre, Apellidos FROM camareros;'

In [136]:
qa.run(p.format(consulta="Lista de los 100 ultimos ticket"))

'SELECT * FROM ticket ORDER BY ID DESC LIMIT 100;'

In [128]:
qa.run(p.format(consulta="Dame la informacion completa de la mesa Z1"))

"SELECT infmesa.*, nommesas.Nombre FROM infmesa INNER JOIN (SELECT UID, Nombre FROM mesas WHERE Nombre = 'Z1') AS nommesas ON infmesa.UID = nommesas.UID;"